In [ ]:
import sys
!{sys.executable} --version
!{sys.executable} -m pip install git+https://github.com/pandas-profiling/pandas-profiling.git

In [ ]:
import pandas as pd
import numpy as np

### Load data, fix problems and merge

In [ ]:
df_bw2017_zbwk = pd.read_csv("./data/bw2017zweitbundestagswahlkreise.csv",sep=';',encoding='latin_1')
df_bw2017_zbwk.head()

In [ ]:
df_bw2017_zst = pd.read_csv("./data/bw2017zweitstadtteile.csv",sep=';',encoding='latin_1')
df_bw2017_zst.head()

In [ ]:
df_bw2017_zwb = pd.read_csv("./data/bw2017zweitwahlbezirke.csv",sep=';',encoding='latin_1')
df_bw2017_zwb.head()

In [ ]:
##trim pre merge
df_bw2017_zst_trimmed = df_bw2017_zst.drop(44) # drop total, we are only interested in local numbers 


df_bw2017_zst_trimmed = df_bw2017_zst_trimmed.drop(columns=['Wahlberechtigte ohne Sperrvermerk',
                               'Wahlberechtigte mit Sperrvermerk',
                               'Wahlberechtigte insgesamt',
                               'Wähler',
                               'darunter Wähler mit Wahlschein',
                               'ungültige Stimmen',
                               'gültige Stimmen',
                               'CDU',
                               'SPD',
                               'GRÜNE',
                               'DIE LINKE',
                               'Die PARTEI',
                               'AfD',
                               'FDP',
                               'PIRATEN',
                               'NPD',
                               'FREIE WÄHLER',
                               'BüSo',
                               'MLPD',
                               'BGE',
                               'DKP',
                               'DM',
                               'ÖDP', 
                               'Tierschutzpartei',
                               'V-Partei³']) #drop total values to only keep relational data

df_bw2017_zst_trimmed = df_bw2017_zst_trimmed.drop(columns=['Anteil Wahlberechtigte mit Sperrvermerk',
                                                            'Anteil Wahlberechtigte insgesamt',
                                                            'Anteil gültige Stimmen']) # redundant infos

# Stadtteilnummer: string -> int
df_bw2017_zst_trimmed['Stadtteilnummer']=df_bw2017_zst_trimmed['Stadtteilnummer'].astype(int)

#convert Strings to Float
for row in list(df_bw2017_zst_trimmed):
    if (row.startswith('Anteil')):
        df_bw2017_zst_trimmed[row] = df_bw2017_zst_trimmed[row].str.replace(',', '.').astype(float)
        
df_bw2017_zst_trimmed.head()

In [ ]:
df_arbeitsmarkt = pd.read_json("./data/arbeitsmarkt.json")
df_arbeitsmarkt.head()

In [ ]:
# drop gesamt
df_arbeitsmarkt.drop(45, inplace=True)    

##split 2011 <-> 2012
l2011 = list()
l2012 = list()
lother = list()
for row in list(df_arbeitsmarkt):
    if row.endswith('2011'):
        l2011.append(row)
    elif row.endswith('2012'):
        l2012.append(row)
    elif row == 'Stadtteil':
        pass
    else:
        lother.append(row)

df_arbeitsmarkt_2011 = df_arbeitsmarkt['Stadtteil'].copy().to_frame()
df_arbeitsmarkt_2011['year'] = 2011
df_arbeitsmarkt_2012 = df_arbeitsmarkt['Stadtteil'].copy().to_frame()
df_arbeitsmarkt_2012['year'] = 2012


for row in l2011:
    df_arbeitsmarkt_2011[row[:-6].strip().replace('\xa0', ' ')] = df_arbeitsmarkt[row].copy()
for row in l2012:
    df_arbeitsmarkt_2012[row[:-6].strip().replace('\xa0', ' ')] = df_arbeitsmarkt[row].copy()
display(df_arbeitsmarkt_2011.head())
display(df_arbeitsmarkt_2012.head())

In [ ]:
# preprocessing arbeitsmarkt
def preprocess(df_old):
    #display(list(df_old))
    #Jeweils Anteil an Gesamtbevölkerung
    df = pd.DataFrame(index=df_old.index)
    df['Stadtteil'] = df_old['Stadtteil']
    df['Erwerbsfähigendichte'] = df_old['Erwerbsfähige Personen Erwerbsfähigendichte in %'].str.replace(',', '.').astype(float)/100
    #Erwerbsfähige Männer / alle
    df['Erwerbsfähige Männer'] = 1.0*df_old['Erwerbsfähige Personen Männer']/df_old['Erwerbsfähige Personen Insgesamt']
    df['Erwerbsfähige Deutsche'] = 1.0*df_old['Erwerbsfähige Personen Deutsche']/df_old['Erwerbsfähige Personen Insgesamt']
    df['Beschäftigtendichte'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Beschäftigtendichte in %'].str.replace(',', '.').astype(float)/100.0
    df['Beschäftigtendichte Männer'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Beschäftigtendichte Männer in %'].str.replace(',', '.').astype(float)/100.0
    df['Beschäftigtendichte Frauen'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Beschäftigtendichte Frauen in %'].str.replace(',', '.').astype(float)/100.0
    df['Beschäftigtendichte Deutsche'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Beschäftigtendichte Deutsche in %'].str.replace(',', '.').astype(float)/100.0
    df['Beschäftigtendichte Ausländer'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Beschäftigtendichte Ausländerinnen und Ausländer in %'].str.replace(',', '.').astype(float)/100.0
    df['Beschäftigtendichte 15-25-Jährige'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Beschäftigtendichte 15- bis unter 25-Jährige in %'].str.replace(',', '.').astype(float)/100.0 
    df['Nebenjobdichte'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Nebenjobdichte in %'].str.replace(',', '.').astype(float)/100.0
    df['Midijobdichte'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Midijobdichte in %'].str.replace(',', '.').astype(float)/100.0
    df['Minijobdichte'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Minijobdichte in %'].str.replace(',', '.').astype(float)/100.0
    df['Minijobdichte_test'] = df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Minijobber/innen']/df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Insgesamt']
    df['Arbeitslosendichte'] = df_old['Arbeitslose Arbeitslosendichte in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte SGB III'] = df_old['Arbeitslose Arbeitslosendichte SGB III in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte SGB II'] = df_old['Arbeitslose Arbeitslosendichte SGB II in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte Frauen'] = df_old['Arbeitslose Arbeitslosendichte Frauen in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte Männer'] = df_old['Arbeitslose Arbeitslosendichte Männer in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte unter 25-Jährige'] = df_old['Arbeitslose Arbeitslosendichte unter 25-Jährige in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte Deutsche'] = df_old['Arbeitslose Arbeitslosendichte Deutsche in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte Ausländer'] = df_old['Arbeitslose Arbeitslosendichte Ausländerinnen und Ausländer in %'].str.replace(',', '.').astype(float)/100.0
    df['Arbeitslosendichte 55- bis unter 65-Jährige'] = df_old['Arbeitslose Arbeitslosendichte 55- bis unter 65-Jährige in %'].str.replace(',', '.').astype(float)/100.0
    return df
df_arbeitsmarkt_2011_preprocessed = preprocess(df_arbeitsmarkt_2011)
df_arbeitsmarkt_2012_preprocessed = preprocess(df_arbeitsmarkt_2012)
df_arbeitsmarkt_2011_preprocessed.head()

In [ ]:
#Problem:
#Wahldaten fassen Bahnhof- und Gutleutviertel zusammen. Um so wenig Daten wie möglich zu verlieren,
#legen wir für die Wahl beide Beobachtungen mit den selben Werten an

def solve_prolem(df_old):
    df = df_old.copy()
    df.drop(df.index[df.Stadtteilname == 'Gutleut-/Bahnhofsviertel'], inplace=True)
    series = df_old[df_old.Stadtteilname == 'Gutleut-/Bahnhofsviertel'].iloc[0,:]
    bahnhof = series.copy()
    bahnhof['Stadtteilname'] = 'Bahnhofsviertel'
    gutleut = series.copy()
    gutleut['Stadtteilname'] = 'Gutleutviertel'
    df = df.append([gutleut,bahnhof])
    return df


df_bw2017_zst_preprocessed = solve_prolem(df_bw2017_zst_trimmed)
df_bw2017_zst_preprocessed.head()


In [ ]:
merge_2011 = df_arbeitsmarkt_2011_preprocessed.set_index('Stadtteil').join(df_bw2017_zst_preprocessed.set_index('Stadtteilname'))
merge_2012 = df_arbeitsmarkt_2012_preprocessed.set_index('Stadtteil').join(df_bw2017_zst_preprocessed.set_index('Stadtteilname'))
merge_2011.to_pickle("./data2011.pkl")
merge_2012.to_pickle("./data2012.pkl")
display(merge_2011.head())
display(merge_2012.head())

### Pandas Profiling
**Achtung:** Die Erstellung des Report dauert Zeit!

In [ ]:
from pandas_profiling import ProfileReport

path2011='report_2011.html'
profile2011 = ProfileReport(merge_2011, title="Pandas Profiling Report: 2011")
profile2011.to_notebook_iframe()

In [ ]:
path2012='report_2012.html'
profile2012 = ProfileReport(merge_2012, title="Pandas Profiling Report: 2012")
profile2012.to_notebook_iframe()

In [ ]:
pd.set_option('display.max_columns', None)
display(df_arbeitsmarkt.head())

In [ ]:
df_arbeitsmarkt_einkommen  = df_arbeitsmarkt['Stadtteil'].copy().to_frame()

for row in lother:
    df_arbeitsmarkt_einkommen[row.strip().replace('\xa0', ' ')] = df_arbeitsmarkt[row].copy()

def preprocess_einkommen(df_old):
    df = pd.DataFrame(index=df_old.index)
    df= pd.DataFrame(index=df_arbeitsmarkt_einkommen.index)
    df['Stadtteil'] = df_old['Stadtteil'] 
    bis_2000 = pd.to_numeric(df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Bruttoarbeitsentgelt bis 2.000 \x80  2010'], errors='coerce')
    von2000bis4000 = pd.to_numeric(df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Bruttoarbeitsentgelt über 2.000 bis 4.000 \x80  2010'], errors='coerce')
    von4000bis5000 = pd.to_numeric(df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Bruttoarbeitsentgelt über 4.000 bis 5.000 \x80  2010'], errors='coerce')
    über5000 = pd.to_numeric(df_old['Sozialversicherungspflichtig Beschäftigte (Wohnort) Bruttoarbeitsentgelt über 5.000 \x80  2010'], errors='coerce')
    gesamt = bis_2000+von2000bis4000+von4000bis5000+über5000 
    df['Anteil Bruttoarbeitsentgelt bis 2.000'] = 1.0*bis_2000/gesamt
    df['Anteil Bruttoarbeitsentgelt über 2.000 bis 4.000'] = 1.0*von2000bis4000/gesamt
    df['Anteil Bruttoarbeitsentgelt über 4.000 bis 5.000'] = 1.0*von4000bis5000/gesamt
    df['Anteil Bruttoarbeitsentgelt über 5.000'] = 1.0*über5000/gesamt
    df['Median Bruttoarbeitsentgelte'] = pd.to_numeric(df_arbeitsmarkt_einkommen['Sozialversicherungspflichtig Beschäftigte (Wohnort) Median Bruttoarbeitsentgelte  2010'], errors='coerce')    
    return df
df_arbeitsmarkt_preprocessed_einkommen = preprocess_einkommen(df_arbeitsmarkt_einkommen)    
    
merge_einkommen = df_arbeitsmarkt_preprocessed_einkommen.set_index('Stadtteil').join(df_bw2017_zst_preprocessed.set_index('Stadtteilname'))
merge_einkommen.head()
# ./data/merged_data.pkl ist pickled Instanz dieses DataFrames

In [ ]:
profile_einkommen = ProfileReport(merge_einkommen, title="Pandas Profiling Report: einkommen")
profile_einkommen.to_notebook_iframe()
#profile_einkommen.to_file('./report_einkommen.html')